In [ ]:
# @title Ячейка 1: Импорт библиотек и настройка окружения
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode

# Модели
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# Метрики и утилиты
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Библиотеки успешно импортированы.")

Библиотеки успешно импортированы.


Подготавливаю данные специально для алгоритма Random Forest:

1.  **Отсутствие масштабирования:** Пропускаю нормализацию, так как деревья решений инвариантны к монотонным преобразованиям признаков. Условие разбиения $x_j \leq t$ не зависит от масштаба (оно работает идентично для $x$ и $1000x$).
2.  **Label Encoding:** Кодирую категории числами. Для ансамблей деревьев это допустимая стратегия: даже если порядок ложный ($0 < 1 < 2$), глубокое дерево способно изолировать категорию $k$ через серию последовательных разбиений.
3.  **Очистка:** Удаляю неинформативные признаки (ID, date) и заполняю пропуски медианой/модой.

In [ ]:
# @title Ячейка 2: Загрузка и подготовка данных

# 1. Загрузка
try:
    df_cls = pd.read_csv('Train.csv')
    df_reg = pd.read_csv('KAG_energydata_complete.csv')
except:
    print("Ошибка: Файлы не найдены.")

# 2. Предобработка (Base)
# Классификация
df_cls_rf = df_cls.copy()
df_cls_rf.drop(['ID'], axis=1, inplace=True, errors='ignore')
# Заполнение пропусков
for col in df_cls_rf.columns:
    if df_cls_rf[col].dtype == 'object':
        df_cls_rf[col] = df_cls_rf[col].fillna(df_cls_rf[col].mode()[0])
    else:
        df_cls_rf[col] = df_cls_rf[col].fillna(df_cls_rf[col].median())
# Кодирование
le_dict = {}
for col in df_cls_rf.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df_cls_rf[col] = le.fit_transform(df_cls_rf[col])
    le_dict[col] = le

X_cls = df_cls_rf.drop('Segmentation', axis=1)
y_cls = df_cls_rf['Segmentation']

X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42, stratify=y_cls
)

# Регрессия
df_reg_rf = df_reg.copy()
df_reg_rf.drop(['date'], axis=1, inplace=True, errors='ignore')
X_reg = df_reg_rf.drop('Appliances', axis=1)
y_reg = df_reg_rf['Appliances']

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

print("Данные подготовлены (Label Encoded). Scaling не требуется.")

Данные подготовлены (Label Encoded). Scaling не требуется.


In [ ]:
# @title Ячейка 3: Обучение бейзлайна (Sklearn RandomForest)

# 1. Classification
print("Training RF Classifier (Baseline)...")
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_clf.fit(X_train_cls, y_train_cls)

# 2. Regression
print("Training RF Regressor (Baseline)...")
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_reg.fit(X_train_reg, y_train_reg)

print("Обучение завершено.")

Training RF Classifier (Baseline)...
Training RF Regressor (Baseline)...
Обучение завершено.


In [ ]:
# @title Ячейка 4: Оценка качества бейзлайна

def evaluate_model(model, X_test, y_test, task='cls'):
    y_pred = model.predict(X_test)
    if task == 'cls':
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        print(f"Accuracy: {acc:.4f}")
        print(f"F1-Score: {f1:.4f}")
        return acc, f1
    else:
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"MAE: {mae:.2f}")
        print(f"R2 Score: {r2:.4f}")
        return mae, r2

print("--- CLASSIFICATION (Baseline RF) ---")
acc_base, f1_base = evaluate_model(rf_clf, X_test_cls, y_test_cls, 'cls')

print("\n--- REGRESSION (Baseline RF) ---")
mae_base, r2_base = evaluate_model(rf_reg, X_test_reg, y_test_reg, 'reg')

# Таблица результатов
results_l4 = {
    'Classification': pd.DataFrame(columns=['Model', 'Accuracy', 'F1']),
    'Regression': pd.DataFrame(columns=['Model', 'MAE', 'R2'])
}
results_l4['Classification'].loc[0] = ['Baseline RF', acc_base, f1_base]
results_l4['Regression'].loc[0] = ['Baseline RF', mae_base, r2_base]

--- CLASSIFICATION (Baseline RF) ---
Accuracy: 0.4950
F1-Score: 0.4930

--- REGRESSION (Baseline RF) ---
MAE: 32.90
R2 Score: 0.5322


# 3. Улучшение бейзлайна: Гипотезы и эксперименты

Случайный лес — это алгоритм бэггинга . Его качество зависит от того, насколько разнообразными некоррелированными получаются деревья в ансамбле.

### a. Формулирование гипотез

1.  **Гипотеза о количестве деревьев (n_estimators):**
    *   Предположение: Увеличение количества деревьев всегда ведет к стабилизации результата и уменьшению дисперсии, не вызывая переобучения (в отличие от бустинга).
    *   План: Проверить значения [50, 100, 200]. Ожидаем рост качества до выхода на плато.

2.  **Гипотеза о глубине деревьев (max_depth):**
    *   Предположение: В бейзлайне деревья строятся до конца. Это в терии может приводить к избыточной сложности отдельных деревьев.

3.  **Гипотеза о признаках (max_features):**
    *   Предположение: главный параметр для борьбы с корреляцией деревьев. Если признаков много, то sqrt работает хорошо. Для регрессии иногда лучше брать все признаки. Подбор этого параметра может дать прирост качества.

4.  **Гипотеза о препроцессинге (OneHotEncoding):**
    *   Предположение: В бейзлайне я использовал LabelEncoding, который вводит ложный порядок (0 < 1 < 2). Для деревьев это не критично, но может создавать неоптимальные разбиения. Использование OneHotEncoding для категориальных признаков с малой кардинальностью (Gender, Married) может помочь лесу строить более логичные правила.

### b. План проверки
я буду использовать GridSearchCV для проверки комбинаций гиперпараметров на данных, обработанных через OneHotEncoder.

In [ ]:
# @title Ячейка 6: Подготовка данных (Improved: OneHot)
# Повторяем логику из ЛР3 для честности сравнения - OHE для категорий

# Классификация
df_cls_imp = df_cls.copy()
df_cls_imp.drop(['ID'], axis=1, inplace=True)
# Fillna
for col in df_cls_imp.columns:
    if df_cls_imp[col].dtype == 'object':
        df_cls_imp[col] = df_cls_imp[col].fillna(df_cls_imp[col].mode()[0])
    else:
        df_cls_imp[col] = df_cls_imp[col].fillna(df_cls_imp[col].median())

# Spending Score Mapping
spending_map = {'Low': 0, 'Average': 1, 'High': 2}
df_cls_imp['Spending_Score'] = df_cls_imp['Spending_Score'].map(spending_map)

# Get dummies for others
X_cls_imp = pd.get_dummies(df_cls_imp.drop('Segmentation', axis=1), drop_first=True)
y_cls_imp = df_cls_imp['Segmentation']

# Split
X_train_cls_imp, X_test_cls_imp, y_train_cls_imp, y_test_cls_imp = train_test_split(
    X_cls_imp, y_cls_imp, test_size=0.2, random_state=42, stratify=y_cls_imp
)

# Регрессия
# Для регрессии RF улучшения препроцессинга особо не нужны (там все числа), но буду тюнить гиперпараметры
X_train_reg_imp, X_test_reg_imp, y_train_reg_imp, y_test_reg_imp = X_train_reg, X_test_reg, y_train_reg, y_test_reg

print("Данные для эксперимента подготовлены.")

Данные для эксперимента подготовлены.


In [ ]:
# @title Ячейка 7: Проверка гипотез (GridSearch)

# Сетка параметров
param_grid = {
    'n_estimators': [100, 200],         # Количество деревьев
    'max_depth': [None, 10, 20],        # Глубина
    'min_samples_leaf': [1, 5],         # Регуляризация листьев
}

# 1. Classification
print("GridSearch Classification (RF)...")
grid_cls = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1)
grid_cls.fit(X_train_cls_imp, y_train_cls_imp)
print(f"Best Params Cls: {grid_cls.best_params_}")

# 2. Regression
print("GridSearch Regression (RF)...")
grid_reg = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, scoring='r2', n_jobs=-1)
grid_reg.fit(X_train_reg_imp, y_train_reg_imp)
print(f"Best Params Reg: {grid_reg.best_params_}")

best_rf_cls = grid_cls.best_estimator_
best_rf_reg = grid_reg.best_estimator_

GridSearch Classification (RF)...
Best Params Cls: {'max_depth': None, 'min_samples_leaf': 5, 'n_estimators': 200}
GridSearch Regression (RF)...
Best Params Reg: {'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 200}


In [ ]:
# @title Ячейка 8: Оценка улучшенных моделей и Выводы

print("\n--- CLASSIFICATION (Improved RF) ---")
acc_imp, f1_imp = evaluate_model(best_rf_cls, X_test_cls_imp, y_test_cls_imp, 'cls')

print("\n--- REGRESSION (Improved RF) ---")
mae_imp, r2_imp = evaluate_model(best_rf_reg, X_test_reg_imp, y_test_reg_imp, 'reg')

results_l4['Classification'].loc[1] = ['Improved RF', acc_imp, f1_imp]
results_l4['Regression'].loc[1] = ['Improved RF', mae_imp, r2_imp]

print("\n--- Сравнение результатов (Baseline vs Improved) ---")
display(results_l4['Classification'])
display(results_l4['Regression'])


--- CLASSIFICATION (Improved RF) ---
Accuracy: 0.5378
F1-Score: 0.5271

--- REGRESSION (Improved RF) ---
MAE: 32.60
R2 Score: 0.5374

--- Сравнение результатов (Baseline vs Improved) ---


,Model,Accuracy,F1
0,Baseline RF,0.495043,0.492965
1,Improved RF,0.537794,0.527055


,Model,MAE,R2
0,Baseline RF,32.900228,0.532236
1,Improved RF,32.602597,0.537383


### g. Выводы по проверке гипотез

1.  **Гипотеза о количестве деревьев (n_estimators):**
    *   Подтвердилась. Лучшие результаты показала модель с максимальным числом деревьев в нашей сетке (200). подтверждает свойство случайного леса: увеличение числа деревьев не приводит к переобучению, а наоборот, уменьшает дисперсию ошибки, делая предсказание более плавным и стабильным.

2.  **Гипотеза о глубине (max_depth):**
    *   Опровергнута (частично). я предполагал, что ограничение глубины поможет. Однако GridSearch выбрал max_depth=None (неограниченная глубина) как для классификации, так и для регрессии.
    *   Интерпретация: В отличие от одиночного дерева, где None ведет к катастрофическому переобучению, в Случайном лесе усреднение ответов сотен глубоких деревьев компенсирует их индивидуальные ошибки. Лес разрешает деревьям быть сложными, так как бэггинг сглаживает результат.

3.  **Гипотеза о Препроцессинге (OneHotEncoding):**
    *   Подтвердилась для классификации. Точность выросла с 49.5% (Baseline с LabelEncoding) до 53.8% (Improved с OneHot). Деревьям стало проще находить разбиения по категориальным признакам (профессия, семейный статус), когда они представлены в бинарном виде.
    *   Нейтральна для регрессии. Метрика R^2 изменилась незначительно (с 0.532 до 0.537), так как в этом датасете основные признаки — числовые (температуры, влажность), на которые кодирование не влияет.


Реализую алгоритм Random Forest на основе метода Bagging, чтобы снизить дисперсию одиночных деревьев.

1.  **Bootstrap:** Для каждого из $N$ эстиматоров создаю случайную подвыборку данных с возвращением ($X_{boot} \subset X$).
2.  **Base Learner:** Обучаю независимые решающие деревья. Для декорреляции моделей использую метод случайных подпространств: каждое дерево при разбиении узла видит только случайное подмножество признаков (обычно $\sqrt{d}$).
3.  **Агрегация:**
    *   **Классификация:** Финальный класс выбирается голосованием большинства (Hard Voting): $\hat{y} = \text{mode}(tree_1(x), \dots, tree_N(x))$.
    *   **Регрессия:** Результат вычисляется как среднее арифметическое предсказаний всех деревьев: $\hat{y} = \frac{1}{N} \sum_{i=1}^{N} tree_i(x)$.

In [ ]:
# @title Ячейка 10: Имплементация MyRandomForest

class MyRandomForest:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_leaf=1, task='classification'):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.task = task
        self.trees = [] # Список для хранения обученных деревьев

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.trees = []
        n_samples = X.shape[0]

        for i in range(self.n_estimators):
            # 1. Bootstrap Sampling
            # Генерируем случайные индексы
            indices = np.random.choice(n_samples, size=n_samples, replace=True)
            X_bootstrap = X[indices]
            y_bootstrap = y[indices]

            # 2. Создаем базовое дерево
            if self.task == 'classification':
                tree = DecisionTreeClassifier(
                    max_depth=self.max_depth,
                    min_samples_leaf=self.min_samples_leaf,
                    max_features='sqrt',
                    random_state=i
                )
            else:
                tree = DecisionTreeRegressor(
                    max_depth=self.max_depth,
                    min_samples_leaf=self.min_samples_leaf,
                    max_features=1.0,
                    random_state=i
                )

            # 3. Обучаем дерево на бутстрап-выборке
            tree.fit(X_bootstrap, y_bootstrap)
            self.trees.append(tree)

    def predict(self, X):
        X = np.array(X)
        # Собираем предсказания от каждого дерева
        tree_preds = np.array([tree.predict(X) for tree in self.trees])

        # Агрегация
        if self.task == 'classification':
            # Голосование большинства (Hard Voting)
            # mode возвращает моду по оси 0 (по деревьям)
            final_preds = mode(tree_preds, axis=0, keepdims=True).mode[0]
        else:
            # Усреднение
            final_preds = np.mean(tree_preds, axis=0)

        return final_preds

print("Класс MyRandomForest (Bagging Logic) реализован.")

Класс MyRandomForest (Bagging Logic) реализован.


In [ ]:
# @title Ячейка 13: Обучение имплементации на улучшеных данных

# Мы должны сравнить нашу реализацию (MyRandomForest) в двух сценариях
# 1. На сырых данных LabelEncoded,чтобы сравнить с Baseline Sklearn
# 2. На улучшенных данных OneHotEncoded + Tuned Params, чтобы сравнить с Improved Sklearn

print("--- 1. Обучение My Impl на RAW данных (Baseline comparison) ---")

# A. Классификация (Raw)
le_y_raw = LabelEncoder()
y_train_cls_raw_enc = le_y_raw.fit_transform(y_train_cls)
y_test_cls_raw_enc = le_y_raw.transform(y_test_cls)

my_rf_clf_raw = MyRandomForest(n_estimators=50, max_depth=None, task='classification')
my_rf_clf_raw.fit(X_train_cls, y_train_cls_raw_enc)

y_pred_my_raw_cls = my_rf_clf_raw.predict(X_test_cls)
acc_my_raw = accuracy_score(y_test_cls_raw_enc, y_pred_my_raw_cls)
f1_my_raw = f1_score(y_test_cls_raw_enc, y_pred_my_raw_cls, average='weighted')

print(f"My RF Raw Accuracy: {acc_my_raw:.4f}")

#  B. Регрессия (Raw)
my_rf_reg_raw = MyRandomForest(n_estimators=50, max_depth=None, task='regression')
my_rf_reg_raw.fit(X_train_reg, y_train_reg)

y_pred_my_raw_reg = my_rf_reg_raw.predict(X_test_reg)
mae_my_raw = mean_absolute_error(y_test_reg, y_pred_my_raw_reg)
r2_my_raw = r2_score(y_test_reg, y_pred_my_raw_reg)

print(f"My RF Raw R2: {r2_my_raw:.4f}")


print("\n--- 2. Обучение My Impl на IMPROVED данных (Improved comparison) ---")

# A. Классификация (Improved
# Данные из Блока 2
le_y_imp = LabelEncoder()
y_train_cls_vec = le_y_imp.fit_transform(y_train_cls_imp)
y_test_cls_vec = le_y_imp.transform(y_test_cls_imp)

# Используем параметры, близкие к найденным в GridSearch
my_rf_clf_imp = MyRandomForest(n_estimators=50, max_depth=None, min_samples_leaf=5, task='classification')
my_rf_clf_imp.fit(X_train_cls_imp, y_train_cls_vec)

y_pred_my_imp_cls = my_rf_clf_imp.predict(X_test_cls_imp)
acc_my_imp = accuracy_score(y_test_cls_vec, y_pred_my_imp_cls)
f1_my_imp = f1_score(y_test_cls_vec, y_pred_my_imp_cls, average='weighted')

print(f"My RF Improved Accuracy: {acc_my_imp:.4f}")

# B. Регрессия (Improved)
# В ЛР4 Improved для регрессии отличался в основном гиперпараметрами
my_rf_reg_imp = MyRandomForest(n_estimators=50, max_depth=None, min_samples_leaf=1, task='regression')
my_rf_reg_imp.fit(X_train_reg_imp, y_train_reg_imp)

y_pred_my_imp_reg = my_rf_reg_imp.predict(X_test_reg_imp)
mae_my_imp = mean_absolute_error(y_test_reg_imp, y_pred_my_imp_reg)
r2_my_imp = r2_score(y_test_reg_imp, y_pred_my_imp_reg)

print(f"My RF Improved R2: {r2_my_imp:.4f}")

# 3. Сохранение в таблицу результатов
# Raw
results_l4['Classification'].loc[2] = ['My Impl. Raw', acc_my_raw, f1_my_raw]
results_l4['Regression'].loc[2] = ['My Impl. Raw', mae_my_raw, r2_my_raw]

# Improved
results_l4['Classification'].loc[3] = ['My Impl. Improved', acc_my_imp, f1_my_imp]
results_l4['Regression'].loc[3] = ['My Impl. Improved', mae_my_imp, r2_my_imp]

print("Все модели обучены и оценены.")

--- 1. Обучение My Impl на RAW данных (Baseline comparison) ---
My RF Raw Accuracy: 0.4913
My RF Raw R2: 0.5214

--- 2. Обучение My Impl на IMPROVED данных (Improved comparison) ---
My RF Improved Accuracy: 0.5266
My RF Improved R2: 0.5284
Все модели обучены и оценены.


In [ ]:
# @title Ячейка 14:  Итоговое сравнение и выводы

print("\n=== ИТОГОВАЯ ТАБЛИЦА: КЛАССИФИКАЦИЯ (LAB 4) ===")
display(results_l4['Classification'])

print("\n=== ИТОГОВАЯ ТАБЛИЦА: РЕГРЕССИЯ (LAB 4) ===")
display(results_l4['Regression'])


=== ИТОГОВАЯ ТАБЛИЦА: КЛАССИФИКАЦИЯ (LAB 4) ===


,Model,Accuracy,F1
0,Baseline RF,0.495043,0.492965
1,Improved RF,0.537794,0.527055
2,My Impl. Raw,0.491326,0.490418
3,My Impl. Improved,0.526642,0.520049



=== ИТОГОВАЯ ТАБЛИЦА: РЕГРЕССИЯ (LAB 4) ===


,Model,MAE,R2
0,Baseline RF,32.900228,0.532236
1,Improved RF,32.602597,0.537383
2,My Impl. Raw,33.278287,0.521363
3,My Impl. Improved,32.982518,0.528422


# 5. Итоговые выводы по Лабораторной работе №4

В работе был исследован метод Random Forest.

**1. Эффективность ансамблирования:**
*   Случайный лес показал наилучшие результаты среди всех рассмотренных ранее алгоритмов (KNN, Linear Models, Decision Tree).
*   В задаче регрессии метрика R^2 достигла 0.537, что три раза выше, чем у линейной регрессии 0.17.

**2. Устойчивость к переобучению:**
*   В отличие от одиночного дерева, случайный лес демонстрирует стабильное качество на тестовой выборке даже при большой глубине деревьев.

**3. Сравнение реализаций:**
*   Собственная реализация алгоритма MyRandomForest для классификации показала результат, близкий к библиотечному (52.6% против 53.7%).
*   В регрессии собственная реализация показала похожий результат ($R^2=0.52$) на библиотечный ($R^2=0.53$). для собственной реализации использовалось всего 50 деревьев, тогда как оптимальное значение, найденное GridSearch для sklearn, составило 200 деревьев. В задачах с высоким уровнем шума количество эстиматоров играет решающую роль.
